## Extract dataset used by a paper with OpenAlex API

Exemple of request for the paper "Exploring Soothness...":  https://api.openalex.org/works/https://doi.org/10.1007/978-3-031-16443-9_4?select=referenced_works

In [13]:
import requests

#Dictionnary of papers with names as key and DOIs as value
papers = {
    "Exploring Smoothness and Class-Separation for Semi-supervised Medical Image Segmentation":"10.1007/978-3-031-16443-9_4",
    "MaxStyle: Adversarial Style Composition for Robust Medical Image Segmentation":"10.1007/978-3-031-16443-9_15",
    "Swin-Unet: Unet-like Pure Transformer for Medical Image Segmentation":"10.1007/978-3-031-25066-8_9",
    "nnFormer: Volumetric Medical Image Segmentation via a 3D Transformer":"10.48550/arXiv.2109.03201",
    "TransUNet: Transformers Make Strong Encoders for Medical Image Segmentation":"10.48550/arXiv.2102.04306"
}

#Dictionnary with dataset names as key and openalex id as value
datasets = {
    "ACDC":"https://openalex.org/W2804047627",
    "LA":"https://openalex.org/W3093394156",
    "PROMISE12":"https://openalex.org/W2106033751",
    "MSCMRSeg":"https://openalex.org/W4312016581",
    "M&Ms":"https://openalex.org/W4226199676",
    "Medical Decathlon":"https://openalex.org/W3172681723",
    "I2CVB":"https://openalex.org/W2049522781",
    "BRATS":"https://openalex.org/W1641498739"
}

In [5]:
#Base url for the request
base_url = "https://api.openalex.org/works/https://doi.org/"

#To only get the field of referenced works in the response
query_param = {
    "select":"referenced_works"
}


#Will be filled with datasets found in the "referenced works" response of each paper
datasets_used = {p:[] for p in papers}

for paper_name in papers:
    request_url = base_url + papers[paper_name]
    request = requests.get(request_url, params=query_param)
    json_response = request.json()
    for ds in datasets:
        if datasets[ds] in json_response['referenced_works']:
            datasets_used[paper_name].append(ds)
datasets_used

{'Exploring Smoothness and Class-Separation for Semi-supervised Medical Image Segmentation': ['ACDC',
  'LA'],
 'MaxStyle: Adversarial Style Composition for Robust Medical Image Segmentation': ['PROMISE12',
  'I2CVB'],
 'nnFormer: Volumetric Medical Image Segmentation via a 3D Transformer': ['ACDC'],
 'Swin-Unet: Unet-like Pure Transformer for Medical Image Segmentation': [],
 'TransUNet: Transformers Make Strong Encoders for Medical Image Segmentation': []}

## Extract papers that use a dataset with OpenAlex

Exemple of request for ACDC: https://api.openalex.org/works?page=2&filter=cites:W2804047627

In [11]:
datasets_used = {p:[] for p in papers}
for dataset in datasets:
    next_page = True
    page_number = 1
    while next_page:
        base_url = "https://api.openalex.org/works"
        query_param = {
            "filter":f"cites:{datasets[dataset]}",
            "page":page_number
        }
        request = requests.get(base_url,params=query_param)
        request_json = request.json()
        
        for res in request_json["results"]:
            if res["title"] in papers:
                datasets_used[res["title"]].append(dataset)
        if not request_json["results"]:
            next_page = False
        else:
            page_number += 1


In [12]:
print(datasets_used)

{'Exploring Smoothness and Class-Separation for Semi-supervised Medical Image Segmentation': ['ACDC', 'LA'], 'MaxStyle: Adversarial Style Composition for Robust Medical Image Segmentation': ['PROMISE12', 'I2CVB'], 'nnFormer: Volumetric Medical Image Segmentation via a 3D Transformer': [], 'Swin-Unet: Unet-like Pure Transformer for Medical Image Segmentation': [], 'TransUNet: Transformers Make Strong Encoders for Medical Image Segmentation': []}
